# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 11:02AM,246701,10,SOTAH0000781,"Nextsource Biotechnology, LLC",Released
1,Sep 19 2022 11:02AM,246701,10,SONSB0001943,"Nextsource Biotechnology, LLC",Released
2,Sep 19 2022 10:30AM,246697,20,ALU0007951,Alumier Labs Inc.,Executing
3,Sep 19 2022 10:30AM,246697,20,ALU0007952,Alumier Labs Inc.,Released
4,Sep 19 2022 10:30AM,246697,20,ALUR016613299,Alumier Labs Inc.,Released
5,Sep 19 2022 10:30AM,246697,20,ALUR024222476,Alumier Labs Inc.,Released
6,Sep 19 2022 10:30AM,246697,20,ALUR079065075,Alumier Labs Inc.,Released
7,Sep 19 2022 10:30AM,246697,20,ALUR091981099,Alumier Labs Inc.,Released
8,Sep 19 2022 10:30AM,246697,20,ALUR096368292,Alumier Labs Inc.,Released
9,Sep 19 2022 10:30AM,246697,20,ALUR155771389,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
3,246677,Executing,1
4,246693,Released,1
5,246697,Executing,2
6,246697,Released,17
7,246701,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246672,3.0,3.0,1.0
246677,NaN,1.0,NaN
246693,NaN,NaN,1.0
246697,NaN,2.0,17.0
246701,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246672,3.0,3.0,1.0
246677,0.0,1.0,0.0
246693,0.0,0.0,1.0
246697,0.0,2.0,17.0
246701,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246672,3,3,1
246677,0,1,0
246693,0,0,1
246697,0,2,17
246701,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246672,3,3,1
1,246677,0,1,0
2,246693,0,0,1
3,246697,0,2,17
4,246701,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246672,3,3,1
1,246677,,1,
2,246693,,,1
3,246697,,2,17
4,246701,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC"
2,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.
21,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America
22,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC"
23,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC",,,2
1,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,,2,17
2,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,,,1
3,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1,
4,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",3,3,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,17,2,
2,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,,
3,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1,
4,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",1,3,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC",2,,
1,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,17,2,
2,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1,,
3,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",,1,
4,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",1,3,3


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
1,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,17.0,2.0,NaN
2,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,NaN,NaN
3,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",NaN,1.0,NaN
4,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",1.0,3.0,3.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 11:02AM,246701,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
1,Sep 19 2022 10:30AM,246697,20,Alumier Labs Inc.,17.0,2.0,0.0
2,Sep 19 2022 9:56AM,246693,16,Sartorius Stedim North America,1.0,0.0,0.0
3,Sep 19 2022 9:36AM,246677,16,"Emersa Waterbox, LLC",0.0,1.0,0.0
4,Sep 19 2022 7:44AM,246672,19,"AdvaGen Pharma, Ltd",1.0,3.0,3.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,246701,2.0,0.0,0.0
16,493370,1.0,1.0,0.0
19,246672,1.0,3.0,3.0
20,246697,17.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,246701,2.0,0.0,0.0
1,16,493370,1.0,1.0,0.0
2,19,246672,1.0,3.0,3.0
3,20,246697,17.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,2.0,0.0,0.0
1,16,1.0,1.0,0.0
2,19,1.0,3.0,3.0
3,20,17.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,2.0
1,16,Released,1.0
2,19,Released,1.0
3,20,Released,17.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Completed,0.0,0.0,3.0,0.0
Executing,0.0,1.0,3.0,2.0
Released,2.0,1.0,1.0,17.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Completed,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,3.0,2.0
2,Released,2.0,1.0,1.0,17.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Completed,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,3.0,2.0
2,Released,2.0,1.0,1.0,17.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()